In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from llama_cpp import Llama
import pymupdf4llm
from langchain.embeddings.base import Embeddings
from langchain_experimental.text_splitter import SemanticChunker
import numpy as np
import json
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
import os
from openai import OpenAI
import re
import torch
import numpy as np
import pandas as pd
from itertools import accumulate

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# file_path = 'question_eval.json' 

# with open(file_path, 'r') as file:
#     data = json.load(file)

In [ ]:
# df = pd.DataFrame(data)
# df.head()

,evaluation,relevance_rating,groundness_rating,standalone_rating,question,answer,context
0,"The question is directly tied to the context, ...",5,5,5,What considerations does the Central Governmen...,In appointing a Controller of Insurance under ...,under section 15K of the Securities and Exchan...
1,"The question is directly tied to the context, ...",5,5,5,What are the conditions under which an insurer...,An insurer can continue to carry on insurance ...,PROVISIONS APPLICABLE TO INSURERS\n 2C. Prohib...
2,The question is highly relevant to the context...,5,4,5,How do the regulations and restrictions on ins...,The regulations and restrictions on insurance ...,annuities of the nature specified in sub-claus...
3,The question is directly related to the contex...,5,4,5,"What provisions of the Insurance Act, 1938 can...",The Central Government has the power to direct...,carry on any class of insurance business in In...
4,The question is relevant to the context as it ...,5,5,5,What are the circumstances under which an insu...,The Authority may suspend or cancel the regist...,debarred by law or practice of that country to...


In [ ]:
# df = df[(df['relevance_rating'] >= 4) & (df['groundness_rating'] >= 4) & (df['standalone_rating'] >= 4)]
# df.shape

(46, 7)

In [4]:
embed_model_path = 'C:/Users/shour/.cache/lm-studio/models/second-state/All-MiniLM-L6-v2-Embedding-GGUF/all-MiniLM-L6-v2-Q4_0.gguf'
embed_model = Llama(model_path='C:/Users/shour/.cache/lm-studio/models/second-state/All-MiniLM-L6-v2-Embedding-GGUF/all-MiniLM-L6-v2-Q4_0.gguf', 
                    embedding=True,
                    verbose=False)

In [32]:
df = pd.read_csv('generations.csv')
df.dropna(inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [33]:
collections = ['semantic', 'char', 'token', 'recursive', 'optimized-semantic', 'optimized-char', 'optimized-token', 'optimized-recursive']
limits = [3, 5]
for collection in collections:
    for limit in limits:
        df[f"{collection}_{limit}"] = None
        df[f"{collection}_{limit}_ctx_len"] = None

In [8]:
llm_client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")

In [9]:
client = QdrantClient(path="rag_eval")

In [10]:
qa_prompt = """
You are tasked with generating a detailed answer for the provided question using only the information given in the context.

Follow these steps:
1. Read the context carefully to understand all relevant details.
2. Read the question and ensure you understand what is being asked.
3. Construct a comprehensive answer that is entirely based on the context. Avoid using external knowledge or assumptions not supported by the context.
4. Ensure that the answer is detailed and fact-based, covering all aspects of the question with sufficient explanation.

Below are some examples (delimited by triple backticks):
# Example 1: 
## Question: "How did the technological advancements during the Industrial Revolution impact urbanization and social structures?"
## Context: "The Industrial Revolution, which took place between the 18th and 19th centuries, brought major technological advancements. Innovations such as the steam engine, mechanized textile production, and improved transportation systems revolutionized manufacturing. These changes led to rapid urbanization, as large numbers of people migrated from rural areas to cities for work, resulting in significant shifts in social structures and labor practices." 
## Generated Answer: "The technological advancements during the Industrial Revolution, including the steam engine and mechanized production, dramatically increased manufacturing efficiency. This efficiency spurred rapid urbanization by drawing large populations from rural areas into cities in search of work. Consequently, traditional social structures were transformed, and new labor practices emerged as a direct result of these urban migrations." 

# Example 2: 
## Question: "What benefits do renewable energy technologies offer, and what challenges must be addressed to ensure their successful implementation?" 
## Context: "Recent developments in renewable energy have underscored the potential of technologies such as solar panels and wind turbines to reduce carbon emissions. Despite these advances, challenges such as efficient energy storage and ensuring a stable power supply during periods of low production continue to hinder widespread adoption. Ongoing research and investment are seen as crucial to overcoming these hurdles and fully realizing the benefits of renewable energy." 
## Generated Answer: "Renewable energy technologies, like solar panels and wind turbines, offer significant benefits including the reduction of carbon emissions and a decreased reliance on fossil fuels. However, for these benefits to be fully realized, challenges such as efficient energy storage and the maintenance of a stable energy supply during low production periods must be addressed. Continued research and investment are key to overcoming these obstacles and ensuring the successful implementation of renewable energy solutions." 

# Example 3: 
## Question: "In what ways has digital technology transformed modern communication?"
## Context: "Digital technology has fundamentally reshaped communication in the modern era. The widespread adoption of the internet and the rise of social media platforms have enabled instant and global exchange of information. These advancements have not only transformed personal communication but also revolutionized business practices, political campaigning, and information dissemination on a global scale." 
## Generated Answer: "Digital technology has transformed modern communication by enabling instantaneous and global information exchange. The adoption of the internet and social media platforms has revolutionized how individuals interact, conduct business, and engage in political processes. These changes have led to more dynamic and interconnected modes of communication, significantly altering traditional communication practices."

Now, similary answer the given question using the context:

Question: \"\"\"{question}\"\"\"
Context: \"\"\"{context}\"\"\"

Please return your answer in the following JSON format:
{{
  "answer": "insert the detailed answer here"
}}
Return Output
"""

In [11]:
p1 = re.compile(
    r"ChatCompletionMessage\(content='.*?\"answer\"\s*:\s*\"(?P<answer>(?:\\.|[^\"\\])*)\"",
    re.DOTALL
)

p2 = re.compile(
    r"arguments='.*?\"answer\"\s*:\s*\"(?P<answer>(?:\\.|[^\"\\])*)\"",
    re.DOTALL
)

In [23]:
def get_index_before_threshold(strings, threshold):
    cumulative_lengths = list(accumulate(len(s) for s in strings))
    print(cumulative_lengths)
    return next((i for i, length in enumerate(cumulative_lengths) if length >= threshold), len(strings)) - 1

In [37]:
for idx, row in df.iterrows():
    search_query = row['question']
    query_vector = embed_model.create_embedding(search_query)['data'][0]['embedding']
    for collection in collections:
        for limit in limits:
            if((df.iloc[idx][f"{collection}_{limit}"] is np.nan) or (df.iloc[idx][f"{collection}_{limit}"] == None)):
                search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
                context = []
                for result in search_result:
                    context.append(result.payload['text'])
                context = [str(chunk) for chunk in context]
                context = '\n'.join(context)
                generation = llm_client.chat.completions.create(
                        model='meta-llama-3.1-8b-instruct',
                        messages=[
                            {
                                "role": "system",
                                "content": """You are an helpful AI assistant. Given a question and related information as context,
                                you have to help the user by answering the question based on the context"""
                            },
                            {
                                "role": "user",
                                "content": qa_prompt.format(
                                    question = search_query,
                                    context = context
                                )
                            }
                        ],
                        tools = [
                            {
                                "type": "function",
                                "function": {
                                    "name": "QuestionAnswering",
                                    "parameters": {
                                        "type": "object",
                                        "title": "QuestionAnswering",
                                        "properties": {
                                            "answer": {
                                                "title": "Answer",
                                                "type": "string",
                                                "description": "A detailed and comprehensive answer based solely on the provided context that fully addresses the question."
                                            }
                                        },
                                        "required": [
                                            "answer"
                                        ]
                                    }
                                }
                            }
                        ],
                        tool_choice={"type": "function", "function": {"name": "QuestionAnswering"}},
                        temperature=0.6,
                    )
                response = generation.choices[0].message
                response = str(response)
                answer = None
                match = p1.search(response)
                if match:
                    answer = match.group("answer")
                elif p2.search(response):
                    match = p2.search(response)
                    answer = match.group("answer")
                else:
                    print(response)
                df.loc[idx, f"{collection}_{limit}"] = answer
                df.loc[idx, f"{collection}_{limit}_ctx_len"] = len(context)

C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='488283514', function=Function(arguments='{"type":"object","properties":{"answer":{"title":"Answer","type":"string","description":"A detailed and comprehensive answer based solely on the provided context that fully addresses the question."}},"required":["answer"]}', name='QuestionAnswering'), type='function')])


C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='766598442', function=Function(arguments='{"type":"object","properties":{"answer":{"title":"Answer","type":"string","description":"A detailed and comprehensive answer based solely on the provided context that fully addresses the question."}},"required":["answer"]}', name='QuestionAnswering'), type='function')])


C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

...

C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='154705174', function=Function(arguments='{"type":"string","value":"The amalgamation process will affect policyholders by merging their policies with the resulting insurance company. Shareholders will see changes in the company\'s ownership structure, and members of each insurer may face changes in their benefits or membership terms. Policyholders\' rights will be protected under the Insurance Act, 1938, which ensures that insurers act in a fair and transparent manner. The Authority may issue directions to settle claims, and policyholders have the right to appeal against any decision made by the insurer. Shareholders\' interests will be safeguarded through the regulatory framework, and members of each insurer will need to adapt to changes in their benefits or membership terms."}', name='QuestionAnswering'), type='function')])


C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='871888323', function=Function(arguments='{"type":"string","value":"The regulations that govern the relocation of an existing place of business in India or outside India are as follows:\\n\\nAccording to Section 2D of the Insurance Act, 1938, every insurer shall be subject to all the provisions of this Act in relation to any class of insurance business so long as his liabilities in India in respect of business of that class remain unsatisfied or not otherwise provided for.\\n\\nSection 4 of the Insurance Act, 1938, states that no person shall begin to carry on any class of insurance business in India and no insurer carrying on any class of insurance business in India shall, after the expiry of three months from the commencement of this Act, continue to carry on any such business, unless he has obtained from the Authority a certificate of regis

C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name=collection, query_vector=query_vector, limit=limit)
C:\Users\shour\AppData\Local\Temp\ipykernel_42876\1653461988.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `que

In [38]:
df.head()

,evaluation,relevance_rating,groundness_rating,standalone_rating,question,answer,context,semantic_3,char_3,token_3,...,optimized-semantic_5_ctx_len,optimized-char_3_ctx_len,optimized-char_5,optimized-char_5_ctx_len,optimized-token_3_ctx_len,optimized-token_5,optimized-token_5_ctx_len,optimized-recursive_3_ctx_len,optimized-recursive_5,optimized-recursive_5_ctx_len
0,"The question is directly tied to the context, ...",5,5,5,What considerations does the Central Governmen...,In appointing a Controller of Insurance under ...,under section 15K of the Securities and Exchan...,When appointing a Controller of Insurance unde...,When appointing a Controller of Insurance unde...,The Central Government appoints a Controller o...,...,5819,11783,The Central Government appoints a Controller o...,19639,3297,When appointing a Controller of Insurance unde...,4972,2642,"When appointing a Controller of Insurance, as ...",6100
1,"The question is directly tied to the context, ...",5,5,5,What are the conditions under which an insurer...,An insurer can continue to carry on insurance ...,PROVISIONS APPLICABLE TO INSURERS\n 2C. Prohib...,An insurer can continue to carry on insurance ...,An insurer can continue to carry on insurance ...,"According to Section 42D of the Insurance Act,...",...,5641,11783,"According to the Insurance Act, 1938, an insur...",19639,2495,An insurer can continue to carry on insurance ...,4064,4648,An insurer can continue to carry on insurance ...,7290
2,The question is highly relevant to the context...,5,4,5,How do the regulations and restrictions on ins...,The regulations and restrictions on insurance ...,annuities of the nature specified in sub-claus...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,...,5385,11999,The Insurance Regulatory and Development Autho...,19999,3206,The Insurance Regulatory and Development Autho...,5763,2642,The regulations and restrictions on insurance ...,6100
3,The question is directly related to the contex...,5,4,5,"What provisions of the Insurance Act, 1938 can...",The Central Government has the power to direct...,carry on any class of insurance business in In...,"The provisions of the Insurance Act, 1938 that...","The Insurance Act, 1938, allows the Central Go...",The Central Government can modify or adapt the...,...,5729,11783,"The Insurance Act, 1938 allows the Central Gov...",19639,3721,"The Insurance Act, 1938 allows the Central Gov...",5212,2338,"The Insurance Act, 1938, provides provisions t...",3990
4,The question is relevant to the context as it ...,5,5,5,What are the circumstances under which an insu...,The Authority may suspend or cancel the regist...,debarred by law or practice of that country to...,An insurer can have their registration suspend...,The registration of an insurer can be suspende...,The Authority may suspend or cancel the regist...,...,5014,6568,An insurer can have their registration suspend...,14454,3205,An insurer’s registration can be suspended or ...,6120,4131,The Authority may suspend or cancel the regist...,6330


In [40]:
df.drop(columns=['evaluation', 'relevance_rating', 'groundness_rating', 'standalone_rating'], inplace=True)

In [41]:
df.head()

,question,answer,context,semantic_3,char_3,token_3,recursive_3,optimized-semantic_3,optimized-char_3,optimized-token_3,...,optimized-semantic_5_ctx_len,optimized-char_3_ctx_len,optimized-char_5,optimized-char_5_ctx_len,optimized-token_3_ctx_len,optimized-token_5,optimized-token_5_ctx_len,optimized-recursive_3_ctx_len,optimized-recursive_5,optimized-recursive_5_ctx_len
0,What considerations does the Central Governmen...,In appointing a Controller of Insurance under ...,under section 15K of the Securities and Exchan...,When appointing a Controller of Insurance unde...,When appointing a Controller of Insurance unde...,The Central Government appoints a Controller o...,The Central Government considers several facto...,When appointing a Controller of Insurance unde...,The Central Government considers the following...,"When appointing a Controller of Insurance, as ...",...,5819,11783,The Central Government appoints a Controller o...,19639,3297,When appointing a Controller of Insurance unde...,4972,2642,"When appointing a Controller of Insurance, as ...",6100
1,What are the conditions under which an insurer...,An insurer can continue to carry on insurance ...,PROVISIONS APPLICABLE TO INSURERS\n 2C. Prohib...,An insurer can continue to carry on insurance ...,An insurer can continue to carry on insurance ...,"According to Section 42D of the Insurance Act,...",An insurer can continue to carry on insurance ...,An insurer can continue to carry on insurance ...,An insurer can continue to carry on insurance ...,An insurer can continue to carry on insurance ...,...,5641,11783,"According to the Insurance Act, 1938, an insur...",19639,2495,An insurer can continue to carry on insurance ...,4064,4648,An insurer can continue to carry on insurance ...,7290
2,How do the regulations and restrictions on ins...,The regulations and restrictions on insurance ...,annuities of the nature specified in sub-claus...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,The Insurance Regulatory and Development Autho...,The regulations and restrictions on insurance ...,The regulations and restrictions on insurance ...,...,5385,11999,The Insurance Regulatory and Development Autho...,19999,3206,The Insurance Regulatory and Development Autho...,5763,2642,The regulations and restrictions on insurance ...,6100
3,"What provisions of the Insurance Act, 1938 can...",The Central Government has the power to direct...,carry on any class of insurance business in In...,"The provisions of the Insurance Act, 1938 that...","The Insurance Act, 1938, allows the Central Go...",The Central Government can modify or adapt the...,"The provisions of the Insurance Act, 1938 that...","The Insurance Act, 1938 provides provisions th...","The Insurance Act, 1938 allows the Central Gov...","The Insurance Act, 1938 allows the Central Gov...",...,5729,11783,"The Insurance Act, 1938 allows the Central Gov...",19639,3721,"The Insurance Act, 1938 allows the Central Gov...",5212,2338,"The Insurance Act, 1938, provides provisions t...",3990
4,What are the circumstances under which an insu...,The Authority may suspend or cancel the regist...,debarred by law or practice of that country to...,An insurer can have their registration suspend...,The registration of an insurer can be suspende...,The Authority may suspend or cancel the regist...,An insurer can have their registration suspend...,An insurer\'s registration can be suspended or...,An insurer can have their registration suspend...,An insurer\'s registration can be suspended or...,...,5014,6568,An insurer can have their registration suspend...,14454,3205,An insurer’s registration can be suspended or ...,6120,4131,The Authority may suspend or cancel the regist...,6330


In [42]:
df.to_csv('generations.csv')